# Deploying Trained Python Models to Fusion

The high level flow for deploying trained python models to Fusion is as follows:

1. Train your model in Python
   * This can be any kind of model (Keras, TF, scikit-learn, etc).
   * When designing your prediction pipeline, be mindful of: (1) input schema, (2) input preprocessing, (3) model prediction, and (4) postprocessing, (5) output schema. For example, for a Keras-based sentiment analysis model --
      * Input schema: A single string value
      * Preprocessing: Tokenization, transform into integer sequence, and zero pad
      * Model prediction: Keras predict(x)
      * Postprocessing: Map output score to "positive" or "negative" label
      * Output schema: float score and string label
   * Serialize any data structures your model needs to disk. Generally, this will be your preprocessing pipeline (like a tokenizer with term->index mappings) and the actual model itself.
2. Create a model bundle ZIP file.  This file contains
   * All serialized pipelines / models that are necessary for prediction
   * `predict.py`:
      * `predict.py`: contains concrete implementations of two functions: init() and predict()
         * `init(bundle_path: str)`: Called once on server startup, used for one-time setup of models, like loading pickled objects, models, etc.
         * `predict(model_input: str)`: Perform model prediction.
            * `model_input` is a dict representing inputs
            * Must return a dict, representing model output. Returned values must be of one of the following types:
              * Any numbers.Number (float, int, etc.)
              * str
              * list or ndarray of str
              * list or ndarray of numbers.Number
3. Test bundle locally using ML service SDK
4. Deploy model to Fusion and test!

This notebook demostrates the process of deploying Python models to Fusion.

In [50]:
BUNDLE_WORKING_DIR = "/tmp/bundle_working_dir"
BUNDLE_ZIP_FILE = "/tmp/mymodel.zip"

!mkdir -p $BUNDLE_WORKING_DIR

## Create `predict.py`

In [51]:
%%writefile $BUNDLE_WORKING_DIR/predict.py

import time

def init(bundle_path: str):
    """
    One-time initialization here.  For example, loading a serialized model from disk.
    
    :param bundle_path: Path to unzipped bundle, used to construct file path to bundle contents, i.e. os.path.join(bundle_path, "model.pkl")
    """
    print(bundle_path)
    time.sleep(1)
    
def predict(model_input: dict) -> dict: 
    """
    Generate prediction.
    
    :param model_input: a dict containing model input
    :return: model output dict
    """
    if 'input' not in model_input:
        raise ValueError("Input must contain the key 'input'")
    
    return {
        "output": model_input['input'], # Just echo
        "vector": [0.9, 1.3, 2, 3, 4, 5]
    }
    

Overwriting /tmp/bundle_working_dir/predict.py


## Create bundle ZIP file

In [52]:
!cd $BUNDLE_WORKING_DIR; zip -r $BUNDLE_ZIP_FILE .

updating: predict.py (deflated 49%)


## Test model locally

In [53]:
from lucidworks.ml.sdk import LocalBundleRunner
runner = LocalBundleRunner(BUNDLE_ZIP_FILE)

/var/folders/dd/rflrpdn974gchxm12nnn2qmh0000gn/T/tmpno9l_tz8


In [54]:
runner.predict({"input": "foo"})

{'output': 'foo', 'vector': [0.9, 1.3, 2, 3, 4, 5]}

## Deploy model to Fusion

In [55]:
from lucidworks.ml.sdk import MLServiceSDKFusionClient
from requests.auth import HTTPBasicAuth

model_id = 'echo'
app_name = 'ml_test'

client = MLServiceSDKFusionClient('http://localhost:6764/api', 
                                  app_name,
                                  auth=HTTPBasicAuth('admin', 'password123'))
client.upload_model(BUNDLE_ZIP_FILE, model_id)

In [56]:
client.list_models()

[{'model_id': 'echo',
  'type': 'python',
  'last_modified': '2019-08-19T14:25:22Z'},
 {'model_id': 'opennlp',
  'type': 'open-nlp',
  'last_modified': '2019-08-19T14:44:34Z'},
 {'model_id': 'spacy',
  'type': 'python',
  'last_modified': '2019-08-19T14:44:33Z'}]

## Test model prediction in Fusion

In [60]:
client.predict(model_id, {"input": "foo"})

{'output': 'foo', 'vector': [0.9, 1.3, 2.0, 3.0, 4.0, 5.0]}

## Generate predictions in a Fusion Query Pipeline

In [58]:
client.create_sample_ml_query_pipeline('sample_echo_ml', model_id, 'input', 'output')

In [61]:
response = client.query('sample_echo_ml', 'ml_test', 'I love this movie')
response

{'response': {'numFound': 0, 'start': 0, 'maxScore': 0.0, 'docs': []},
 'responseHeader': {'zkConnected': True,
  'status': 0,
  'QTime': 1,
  'params': {'output': 'I love this movie',
   'q': 'I love this movie',
   'defType': 'edismax',
   'lw.pipelineId': 'sample_echo_ml',
   'fl': ['*', 'score'],
   'context': 'app:ml_test',
   'start': '0',
   'isFusionQuery': 'true',
   'rows': '10',
   'wt': 'json',
   'fusionQueryId': 'mI2G07iTt7'},
  'totalTime': 182}}